In [ ]:
#@markdown <center><h1>Install</h1></center>

%cd /content
!git clone -b totoro6 https://github.com/LucipherDev/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

In [ ]:
#@markdown <center><h1>Load Models</h1></center>

import torch
from nodes import NODE_CLASS_MAPPINGS

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()

flux_version = "dev" # @param ["dev","schnell"]

print(f"Downloading Flux.1-{flux_version}...")

if flux_version == "schnell":
  !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.safetensors -d /content/TotoroUI/models/unet -o flux1-schnell.safetensors
elif flux_version == "dev":
  !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev.safetensors

print("Downloading VAE...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft

print("Downloading Clips...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

with torch.inference_mode():
    print("Loading VAE...")
    vae = VAELoader.load_vae("ae.sft")[0]
    print(f"Loading Flux.1-{flux_version}...")
    unet = UNETLoader.load_unet(f"flux1-{flux_version}.safetensors", "fp8_e4m3fn")[0]
    print("Loading Clips...")
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]

    unet_f, clip_f = unet, clip

print("All Models Loaded!")

In [ ]:
# @markdown <center><h1>Functions</h1></center>

import re
import os
import gc
import random
import numpy as np
from PIL import Image
from google.colab import files

import nodes
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro_extras import nodes_flux
from totoro import model_management

CLIPTextEncodeFlux = nodes_flux.NODE_CLASS_MAPPINGS["CLIPTextEncodeFlux"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()


loras = {
    "xlabs_flux_anime":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/anime_lora_comfy_converted.safetensors",
         "filename": "xlabs_anime_lora.safetensors",
         "triggers": "anime"
         },
    "xlabs_flux_art":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/art_lora_comfy_converted.safetensors",
         "filename": "xlabs_art_lora.safetensors",
         "triggers": "art"
         },
    "xlabs_flux_disney":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/disney_lora_comfy_converted.safetensors",
         "filename": "xlabs_disney_lora.safetensors",
         "triggers": "disney style"
         },
    "xlabs_flux_mjv6":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/mjv6_lora_comfy_converted.safetensors",
         "filename": "xlabs_mjv6_lora.safetensors",
         },
    "xlabs_flux_realism":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/realism_lora_comfy_converted.safetensors",
         "filename": "xlabs_realism_lora.safetensors",
         },
    "xlabs_flux_scenery":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/scenery_lora_comfy_converted.safetensors",
         "filename": "xlabs_scenery_lora.safetensors",
         "triggers": "scenery style"
         },
    # "xlabs_flux_furry":
    #  {
    #      "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/furry_lora.safetensors",
    #     "filename": "xlabs_flux_furry_lora.safetensors",
    #     },
}

def load_loras(prompt):
  # @markdown <ul><li><h2>Load Loras</h2>- Add &lt;lora_name:model_strength&gt; to Prompt</li></ul>

  global unet, clip, unet_f, clip_f

  unet_f, clip_f = unet, clip

  matches = re.findall(r"<\s*([^:]+?)\s*:\s*([0-9.]+)\s*>", prompt)

  loras_list = [(name.strip(), float(value)) for name, value in matches]

  if len(loras_list):
    print("Loading Loras...")

  for lora_tuple in loras_list:
    lora = loras.get(lora_tuple[0], None)

    if lora:
      !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M {lora["url"]} -d /content/TotoroUI/models/loras -o {lora["filename"]}

      with torch.inference_mode():
        unet_f, clip_f = LoraLoader.load_lora(unet_f, clip_f, lora["filename"], lora_tuple[1], lora_tuple[1])

      print(f"Loaded Lora: {lora_tuple[0]}")
    else:
      print(f"Lora not listed: {lora_tuple[0]}")

def clean_prompt(prompt):
  cleaned_prompt = re.sub(r"<.*?>", "", prompt)

  return cleaned_prompt

def cuda_gc():
  try:
    model_management.soft_empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
  except:
    pass

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

def save_image(decoded, path, name, download=False):
  full_path = os.path.abspath(os.path.join(path, name))
  Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save( full_path)

  img = Image.open(full_path)
  display(img)

  if download:
    files.download(full_path)

@torch.inference_mode()
def generate(prompt, width, height, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download, mode="t2i", input_img=None, denoise=1.0):
  global unet, clip, unet_f, clip_f

  print("Prompt Received")

  load_loras(prompt)
  prompt = clean_prompt(prompt)

  if mode == "t2i":
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

  elif mode == "i2i":
    image = nodes.LoadImage().load_image(input_img)[0]
    latent_image = ImageScaleToTotalPixels.upscale(image, "lanczos", 1.0)[0]
    latent_image = VAEEncode.encode(vae, latent_image)[0]

  cond = CLIPTextEncodeFlux.encode(clip_f, prompt, prompt, guidance)[0]
  guider = BasicGuider.get_guider(unet_f, cond)[0]
  sampler = KSamplerSelect.get_sampler(sampler_name)[0]
  sigmas = BasicScheduler.get_sigmas(unet_f, scheduler, steps, denoise)[0]

  for i in range(0, batch_size):
    if fixed_seed == 0:
      seed = random.randint(0, 18446744073709551615)
    else:
      seed = fixed_seed

    print("Seed:", seed)

    noise = RandomNoise.get_noise(seed)[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    save_image(decoded, "/content", f"flux_{mode}_{seed}_{i}.png", auto_download)

  cuda_gc()

print(f"{'Lora Name':<40} {'Trigger Words':<40}")
print("-" * 80)

for key_name, details in loras.items():
    trigger_words = details.get("triggers", "N/A")
    print(f"{key_name:<40} {trigger_words:<40}")

In [ ]:
#@markdown <center><h1>Txt2Img</h1></center>

positive_prompt = "black forest toast spelling out the words 'FLUX DEV', tasty, food photography, dynamic shot" # @param {"type":"string"}
width = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
height = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
guidance = 3.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
steps = 20 # @param {"type":"slider","min":4,"max":50,"step":1}
sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
auto_download = False # @param {"type":"boolean"}

generate(positive_prompt, width, height, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download)

In [ ]:
#@markdown <center><h1>Img2Img</h1></center>

positive_prompt = "anime style" # @param {"type":"string"}
fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
guidance = 3.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
steps = 20 # @param {"type":"slider","min":4,"max":50,"step":1}
sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
input_img = "/content/test.png" # @param {"type":"string"}
denoise = 0.85 # @param {"type":"slider","min":0,"max":1,"step":0.01}
batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
auto_download = False # @param {"type":"boolean"}


generate(positive_prompt, 0, 0, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download, "i2i", input_img, denoise)